In [ ]:
%pip install sagemath

In [ ]:
from sage.all import GF, PolynomialRing, ZZ, matrix, vector, randint, choice
from IPython.display import display, Latex, Markdown
import random



# Baby Kyber https://cryptopedia.dev/posts/kyber/


## Parameter

Alle Parameter können frei angepasst werden.
Die Standardwerte sollten zu übersichtlichkeit klein gehalten werden.


In [ ]:
randomParameter = False

# ===== Parameter =====
q = 17              # Modulus für Koeffizienten
n = 4               # Grad: Polynomring Z_q[x] / (x^n + 1)
k = 2               # Vektorlänge
kleine_faktoren = [16, 0, 1]  # -1 ≡ 16 (mod 17), 0, 1

q_halbe = (q + 1) // 2

print("q =", q)
print("n =", n)
print("k =", k)
print("⌊q/2⌉ =", q_halbe)



## Polynom-Arithmetik

Polynome werden als Listen dargestellt:
[a0, a1, a2, a3]  ↔  a0 + a1·x + a2·x² + a3·x³


In [ ]:
# Galois Field GF(17)
GF17 = GF(q)

# Polynomring Z_q[x] über GF(q)
Rx.<x> = PolynomialRing(GF17)

# Quotientenring Z_q[x] / (x^n + 1)
# Dies ist unser Polynomring mit automatischer Reduktion modulo (x^n + 1)
Q.<x> = Rx.quotient(x^n + 1)

# Kleine Faktoren für zufällige Polynome
kleine_faktoren = [GF17(f) for f in [16, 0, 1]]  # -1 ≡ 16 (mod 17), 0, 1

def zufalls_polynom(klein=True):
    """Generiere zufälliges Polynom mit Koeffizizienten in GF(q)"""
    if klein:
        coeffs = [choice(kleine_faktoren) for _ in range(n)]
    else:
        coeffs = [GF17(randint(0, q-1)) for _ in range(n)]
    return Q(Rx(coeffs))

def poly_str(p):
    """Formatiert ein Polynom als lesbare String mit x^i Notation"""
    if isinstance(p, list):
        # Falls es eine Liste ist - bereits in aufsteigender Reihenfolge
        coeffs = p
    else:
        # SageMath Polynom - konvertiere zu Coefficients
        try:
            coeffs = list(p.list())  # Gibt Koeffizienten in aufsteigender Reihenfolge zurück
        except:
            coeffs = [p]
    
    terms = []
    for i, coeff in enumerate(coeffs):
        coeff_int = int(coeff) % q
        if coeff_int == 0:
            continue
        
        if i == 0:
            # Konstanter Term
            terms.append(str(coeff_int))
        elif i == 1:
            # x^1
            if coeff_int == 1:
                terms.append("x")
            else:
                terms.append(f"{coeff_int}·x")
        else:
            # x^i für i >= 2
            if coeff_int == 1:
                terms.append(f"x^{i}")
            else:
                terms.append(f"{coeff_int}·x^{i}")
    
    if not terms:
        return "0"
    # Rückwärts anzeigen (höchste Potenz zuerst)
    return " + ".join(reversed(terms))

def matrix_latex(M):
    """Formatiert eine Matrix als LaTeX und gibt Latex-Objekt zurück"""
    rows = []
    for row in M:
        row_str = " & ".join([poly_str(p) for p in row])
        rows.append(row_str)
    matrix_str = " \\\\ ".join(rows)
    latex_str = f"\\begin{{pmatrix}} {matrix_str} \\end{{pmatrix}}"
    return Latex(latex_str)

def vector_latex(v):
    """Formatiert einen Vektor als LaTeX und gibt Latex-Objekt zurück"""
    row_str = " \\\\ ".join([poly_str(p) for p in v])
    latex_str = f"\\begin{{pmatrix}} {row_str} \\end{{pmatrix}}"
    return Latex(latex_str)

def poly_latex(v):
    """Formatiert ein Polynom als zentriertes LaTeX-Objekt"""
    latex_str = r"\[ " + str(v) + r" \]"
    return Latex(latex_str)



## Schlüsselgenerierung durch Alice


In [ ]:
# Privater Schlüssel s
if randomParameter:
    s = [zufalls_polynom(klein=True) for _ in range(k)]
else:
    # s=(−x3−x2+x,−x3−x)
    # Koeffizientenreihenfolge AUFSTEIGEND: [a0, a1, a2, a3] = [const, x, x², x³]
    s = [Q(Rx([GF17(0), GF17(1), GF17(16), GF17(16)])), 
         Q(Rx([GF17(0), GF17(16), GF17(0), GF17(16)]))]

# Fehlervektor e
if randomParameter:
    e = [zufalls_polynom(klein=True) for _ in range(k)]
else:
    # e=(x2,x2−x)
    e = [Q(Rx([GF17(0), GF17(0), GF17(1), GF17(0)])), 
         Q(Rx([GF17(0), GF17(16), GF17(1), GF17(0)]))]

# Öffentliche Matrix A
if randomParameter:
    A = [[zufalls_polynom(klein=False) for _ in range(k)] for _ in range(k)]
else:
    # A = [[6x³+16x²+16x+11, 9x³+4x²+6x+3],
    #      [5x³+3x²+10x+1,  6x³+x²+9x+15]]
    A = [[Q(Rx([GF17(11), GF17(16), GF17(16), GF17(6)])), 
          Q(Rx([GF17(3), GF17(6), GF17(4), GF17(9)]))],
         [Q(Rx([GF17(1), GF17(10), GF17(3), GF17(5)])), 
          Q(Rx([GF17(15), GF17(9), GF17(1), GF17(6)]))]]
    
display(Markdown("**Zufällig generierter Privater Schlüssel s:**"))
display(vector_latex(s))

display(Markdown("**Zufällig generierter Fehlervektor e:**"))
display(vector_latex(e))

display(Markdown("**Zufällig generierte Öffentliche Matrix A:**"))
display(matrix_latex(A))

# t = A*s + e (automatisch modulo (x^n + 1) dank Quotientenring)
t = []
for i in range(k):
    t_i = Q(0)
    for j in range(k):
        t_i = t_i + A[i][j] * s[j]
    t_i = t_i + e[i]
    t.append(t_i)

display(Markdown(r"**Öffentlicher Vektor t**"))
display(Markdown(r"\\[ \mathbf{t} = A\mathbf{s} + \mathbf{e} \\]"))
display(vector_latex(t))



## Verschlüsselung durch Bob


In [ ]:
# Zufälliger Vektor r und Fehler
if randomParameter:
    r = [zufalls_polynom(klein=True) for _ in range(k)]
else: 
    # r = (-x³ + x², x³ + x² - 1)
    r = [Q(Rx([GF17(0), GF17(0), GF17(1), GF17(16)])), 
         Q(Rx([GF17(16), GF17(0), GF17(1), GF17(1)]))]

if randomParameter:
    e1 = [zufalls_polynom(klein=True) for _ in range(k)]
else:
    # e1 = (x² + x, x²)
    e1 = [Q(Rx([GF17(0), GF17(1), GF17(1), GF17(0)])), 
          Q(Rx([GF17(0), GF17(0), GF17(1), GF17(0)]))]

if randomParameter:
    e2 = zufalls_polynom(klein=True)
else:
    # e2 = -x³ - x²
    e2 = Q(Rx([GF17(0), GF17(0), GF17(16), GF17(16)]))
    
display(Markdown("**Zufallsvektor r:**"))
display(vector_latex(r))

display(Markdown("**Zufällig generierter Fehler e1:**"))
display(vector_latex(e1))

display(Markdown("**Zufällig generierter Fehler e2**:"))
display(poly_str(e2))

# Nachricht (zufällige 4 Bit)
if randomParameter:
    m_bin = [random.randint(0,1) for _ in range(n)]
else:
    # m_bin entspricht 11 in binary = 1011, als Polynom x³ + x + 1
    m_bin = [1, 1, 0, 1]

m_coeffs = [GF17(q_halbe * x) for x in m_bin]
m = Q(Rx(m_coeffs))

display(Markdown("**Binäre Nachricht m_bin:**"))
display(poly_str(m_bin))
display(Markdown("**Skalierte Nachricht m:**"))
display(poly_str(m))

# u = A^T r + e1 (automatisch modulo (x^n + 1))
u = []
for j in range(k):
    u_j = Q(0)
    for i in range(k):
        u_j = u_j + A[i][j] * r[i]
    u_j = u_j + e1[j]
    u.append(u_j)

# v = t^T r + e2 + m (automatisch modulo (x^n + 1))
v = Q(0)
for i in range(k):
    v = v + t[i] * r[i]
v = v + e2 + m

display(Markdown("**Ciphertext u:**"))
display(Markdown("$\\mathbf{u} = \\mathbf{A}^T \\mathbf{r} + \\mathbf{e}_1$"))
display(vector_latex(u))

display(Markdown("**Ciphertext v**:"))
display(Markdown("$v = \\mathbf{t}^T \\mathbf{r} + e_2 + m$"))
display(poly_str(v))



## Entschlüsselung durch Alice


In [ ]:
# m_noisy = v - s^T u (automatisch modulo (x^n + 1))
m_noisy = v
for i in range(k):
    m_noisy = m_noisy - s[i] * u[i]

display(Markdown("**Rauschbehaftete Nachricht m~:**"))
display(Markdown("$m_{\\text{noisy}} = v - \\mathbf{s}^T \\mathbf{u}$"))
display(poly_str(m_noisy))

# Rundung
# Extrahiere Koeffizienten aus dem SageMath-Polynom
m_noisy_coeffs = list(m_noisy.lift().coefficients(sparse=False))

# Pad mit Nullen falls nötig
while len(m_noisy_coeffs) < n:
    m_noisy_coeffs.append(GF17(0))

m_rec = []
for coeff in reversed(m_noisy_coeffs[-n:]):
    c = int(coeff)
    if abs(c - q_halbe) < abs(c):
        m_rec.append(1)
    else:
        m_rec.append(0)

display(Markdown("**Wiederhergestellte Bits (nach Rundung):**"))
display(poly_str(m_rec))
display(Markdown("**Originale Bits:**"))
display(poly_str(m_bin))

# Vergleich
if m_rec == m_bin:
    display(Markdown("✓ **Erfolgreich dekodiert!**"))
else:
    display(Markdown("✗ **Dekodierung fehlgeschlagen**"))



## Zusammenfassung

- Alle Werte zufällig erzeugt
- Jeder Zwischenschritt ausgegeben
- Klassische Baby-Kyber-Darstellung
- Ideal zum Nachrechnen auf Papier

So wurde Kryptographie traditionell erklärt: klein, offen, nachvollziehbar.
